#### Prompt templates

prompt  templates help to turn raw user information into a format that the llm can work with. In the case, the raw user input is just a message, which we are passing to the llm. let's now make that a bit more complicated. First, let's add in a system message with some custom instructions(but still taking messages as input). Next, we'll add in more input besides just the messages.


In [16]:
import os
from rich import print
from dotenv import load_dotenv 
load_dotenv(dotenv_path="../.env")
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
llm=ChatGroq(model="gemma2-9b-it")

In [10]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.messages import HumanMessage
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",("You are helpful assistant.Answer all the questions to the best of your ability.")),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|llm

In [8]:
chain

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [18]:
print(chain.invoke({"messages":[HumanMessage(content="what is Agentic ai ?")]}).content)

Let me explain Agentic AI. 

**Agentic AI** refers to artificial intelligence systems that exhibit **autonomous decision-making and 
goal-oriented behavior**.  

Think of an agent like a little program with its own goals and the ability to act in the world to achieve them. 

Here's a breakdown of key characteristics:

* **Goal-Oriented:**  Agentic AI systems are designed with specific objectives in mind. They work towards 
accomplishing these goals, even adapting their strategies as needed.
* **Autonomous:**  They can make independent decisions and take actions without constant human intervention. They 
learn from their environment and experiences to improve their performance.
* **Reactive:** Agentic AI responds to changes in its surroundings. If something unexpected happens, it can adjust 
its plans accordingly.

**Examples of Agentic AI:**

* **Self-driving cars:**  They navigate roads, make decisions about speed and lane changes, and avoid obstacles, 
all while aiming to reach their destination safely.
* **Chatbots:**  Advanced chatbots can hold natural conversations, understand user requests, and provide helpful 
responses.
* **Game AI:**  Non-player characters (NPCs) in video games often exhibit agentic behavior, making decisions about 
their actions based on the game's rules and their goals.

**Important Considerations:**

* **Ethics:**  As agentic AI becomes more powerful, it's crucial to consider the ethical implications of their 
actions. Who is responsible when an AI makes a mistake? How do we ensure they act in a safe and beneficial way?
* **Safety:**  Designing reliable and safe agentic AI is essential.  We need to prevent unintended consequences and
ensure they don't pose risks to humans or the environment.


Let me know if you have any other questions about Agentic AI.

In [19]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]



In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history) ## added chain 

In [22]:
config={"configurable":{"session_id":"chat1"}}

In [24]:
response=with_message_history.invoke(
    [HumanMessage(content="hi my name is abhijit")],
                                      config=config)

In [ ]:
print(response)

AIMessage(
    content="Hi Abhijit, it's nice to meet you! \n\nI'm happy to answer any questions you have. Just ask away! 😊  
\n\n",
    additional_kwargs={},
    response_metadata={
        'token_usage': {
            'completion_tokens': 35,
            'prompt_tokens': 32,
            'total_tokens': 67,
            'completion_time': 0.063636364,
            'prompt_time': 0.0014751,
            'queue_time': 0.24945953,
            'total_time': 0.065111464
        },
        'model_name': 'gemma2-9b-it',
        'system_fingerprint': 'fp_10c08bf97d',
        'service_tier': 'on_demand',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--f5aeec8a-08d4-4604-b188-235694a6f603-0',
    usage_metadata={'input_tokens': 32, 'output_tokens': 35, 'total_tokens': 67}
)

In [26]:
# updating propmt

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",("You are helpful assistant.Answer all the questions to the best of your ability in {language}")),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt | llm


In [27]:
response=chain.invoke({"messages":[HumanMessage(content="hi my name is krish ")],"language":"telugu"})

In [29]:
print(response.content)

మీకు నమస్కారం కృష్ణ! 😊 

నేను మీకు సహాయకరమైన సహచరునిగా ఉన్నాను. ఏ ప్రశ్నైనా అడిగండి, నేను నా జ్ఞానానికి అనుగుణంగా మీకు సమాధానం ఇస్తాను.

Let's now wrap this more complicated chain in a message history class. this time , because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [30]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [31]:
config={"configurable":{"session_id":"chat4"}}

response=with_message_history.invoke(
    {'messages':[HumanMessage(content="hi, my name is abhijit and i am ai engineer")],"language":"telugu"},
    config=config
)

In [32]:
print(response)

AIMessage(
    content='నేను మీకు సహాయకరి అని కూడా చెప్పవచ్చు. మీ పేరు Abhijit అని తెలుసుకున్నాను, అలాగే మీరు AI ఇంజనీర్ అని కూడా తెలుసుకున్నాను.\n\nమీరు ఏమి అడిగితే అది
తెలుగులోనే సమాధానం ఇస్తానని గుర్తుంచుకోండి. 😊\n\nప్రశ్నలు అడిగి మాట్లాడుకోండి! 🤝\n\n',
    additional_kwargs={},
    response_metadata={
        'token_usage': {
            'completion_tokens': 123,
            'prompt_tokens': 40,
            'total_tokens': 163,
            'completion_time': 0.223636364,
            'prompt_time': 0.00155246,
            'queue_time': 0.24962632,
            'total_time': 0.225188824
        },
        'model_name': 'gemma2-9b-it',
        'system_fingerprint': 'fp_10c08bf97d',
        'service_tier': 'on_demand',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--5afda680-1998-4820-b344-495e3371502c-0',
    usage_metadata={'input_tokens': 40, 'output_tokens': 123, 'total_tokens': 163}
)